# Tokenize and save data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

In [3]:
import pandas as pd

## Preparing the data

In [4]:
source_dir = 'midi_transcribe_v1_simple'
path = Path(f'data/midi/{source_dir}/')
csv_path = Path(f'data/midi/metadata/{source_dir}.csv')
path.ls()[:5]

[PosixPath('data/midi/midi_transcribe_v1_simple/ecomp'),
 PosixPath('data/midi/midi_transcribe_v1_simple/classic_piano'),
 PosixPath('data/midi/midi_transcribe_v1_simple/cprato'),
 PosixPath('data/midi/midi_transcribe_v1_simple/freemidi'),
 PosixPath('data/midi/midi_transcribe_v1_simple/midiworld')]

### Select file data

In [5]:
cache_name = 'tmp_classical'

In [6]:
csv = pd.read_csv(csv_path)
csv = csv.loc[csv[source_dir].notna()];

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,12,14,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
sv = csv.seconds.values
sv = sv[~np.isnan(sv)]
sv.sort()
sv[-100:]

array([ 1551.452991,  1556.324786,  1572.264957,  1577.820513, ...,  3012.571429,  3126.908425,  3126.908425,
       23520.961538])

In [8]:
files = csv[source_dir]

In [9]:
flist = [Path(f) for f in files.values]
flist = [f for f in flist if f.exists()]

### Create databunch

In [10]:
bs=32

In [11]:
class MusicTokenizer():
    def __init__(self):
        super().__init__()
        self.n_cpus = num_cpus()
        
    def process_text(self, t:str) -> List[str]:
        return t.split(" ")
    
    def _process_all_1(self, texts:Collection[str]) -> List[List[str]]:
        return [self.process_text(t) for t in texts]

    def process_all(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts`."
        if self.n_cpus <= 1: return self._process_all_1(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            return sum(e.map(self._process_all_1, partition_by_cores(texts, self.n_cpus)), [])


In [12]:
def lm_join_texts(texts:Collection[str]):
    return [f'{BOS} {t}' for t in texts]

In [13]:
class LMOpenFileProcessor(PreProcessor):
    "`PreProcessor` that opens the filenames and read the texts."
    def process_one(self,item): return open_text(item) if isinstance(item, Path) else item
    def process(self, ds:Collection): ds.items = [self.process_one(item) for item in ds.items] # converting array gets OOM error

In [14]:
class LMTokenizeProcessor(PreProcessor):
    "`PreProcessor` that tokenizes the texts in `ds`."
    def __init__(self, ds:ItemList=None, tokenizer:Tokenizer=None, chunksize:int=10000):
        self.tokenizer,self.chunksize = ifnone(tokenizer, Tokenizer()),chunksize

    def process_one(self, item):  return self.tokenizer._process_all_1([item])[0]
    def process(self, ds):
        ds.items = lm_join_texts(ds.items)
        tokens = []
        for i in progress_bar(range(0,len(ds),self.chunksize), leave=False):
            tokens += self.tokenizer.process_all(ds.items[i:i+self.chunksize])
        ds.items = tokens

In [15]:
# Debugging

In [15]:
ps = [LMOpenFileProcessor(), 
     LMTokenizeProcessor(tokenizer=MusicTokenizer()),#, chunksize=72*20),
     NumericalizeProcessor(vocab=None, max_vocab=500)]

In [16]:
data = TextList(items=flist, path=path, processor=ps)

In [17]:
data_split = data.random_split_by_pct(0.05, seed=6)

In [18]:
data_lm = data_split.label_for_lm()

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [22]:
databunch = data_lm.databunch(bs=bs)

In [24]:
databunch.one_batch()

(tensor([[37,  8,  9,  ...,  8, 16,  8],
         [ 8, 46, 10,  ..., 14,  8, 19],
         [ 8, 47, 10,  ..., 15,  8, 22],
         ...,
         [24,  8,  9,  ..., 10, 11, 10],
         [25,  8, 30,  ...,  9, 12, 10],
         [ 9, 12,  8,  ..., 11, 10, 13]]),
 tensor([[ 8,  9, 17,  ..., 16,  8, 22],
         [46, 10,  9,  ...,  8, 19,  8],
         [47, 10,  9,  ...,  8, 22,  8],
         ...,
         [ 8,  9, 12,  ..., 11, 10, 13],
         [ 8, 30,  8,  ..., 12, 10, 11],
         [12,  8, 11,  ..., 10, 13, 10]]))

In [32]:
#     data.save(cache_name)
vocab = databunch.train_ds.vocab
len(databunch.train_ds), len(databunch.valid_ds), len(databunch.train_ds.vocab.itos)

(5, 5, 8)

In [9]:
if (path/f'{cache_name}/itos.pkl').exists():
# if False:
    data = TextLMDataBunch.load(path, bs=bs)
else:
    p = [OpenFileProcessor(), TokenizeProcessor(tokenizer=MusicTokenizer(), chunksize=10), NumericalizeProcessor(vocab=None, max_vocab=500)]

    data = (TextList.from_folder(path, recurse=True, processor=p)
            .random_split_by_pct(0.05, seed=6)
            .label_for_lm()
            .databunch(bs=bs))
    data.save(cache_name)
vocab = data.train_ds.vocab
len(data.train_ds), len(data.valid_ds), len(data.train_ds.vocab.itos)

MemoryError: 

In [10]:
%debug

> /home/ubuntu/fastai/fastai/core.py(262)array()
    260     if np.int_==np.int32 and dtype is None and is_listy(a) and len(a) and isinstance(a[0],int):
    261         dtype=np.int64
--> 262     return np.array(a, dtype=dtype, **kwargs)
    263 
    264 class EmptyLabel(ItemBase):

ipdb> len(a)
28151
ipdb> len(a[0])
105025
ipdb> a[0][0]
--KeyboardInterrupt--
ipdb> quit


In [ ]:
t = data.train_ds[0][0]
t.text[:50], t.data

### Testing

In [8]:
data.show_batch()

idx,text
0,xxbos nF o3 t1 i0 nA o3 t1 i0 nC o4 t1 i0 || |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0 || |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0 || |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0 || |e| |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0
1,i0 nC o3 t2 i0 || |e| |s| nF o2 t2 i0 nA o2 t2 i0 nC o3 t2 i0 || |s| nF o2 t2 i0 nA o2 t2 i0 nC o3 t2 i0 || |s| nF o2 t2 i0 nA o2 t2 i0 nC o3 t2 i0 || |s| nF o2 t2 i0 nA o2 t2 i0 nC o3 t2 i0 || |e| |s| nF o2 t2 i0 nA
2,o3 t2 i0 nE o3 t2 i0 nG o3 t2 i0 || |e| |s| nC o3 t2 i0 nE o3 t2 i0 nG o3 t2 i0 || |s| nC o3 t2 i0 nE o3 t2 i0 nG o3 t2 i0 || |s| nC o3 t2 i0 nE o3 t2 i0 nG o3 t2 i0 || |s| nC o3 t2 i0 nE o3 t2 i0 nG o3 t2 i0 || |e|
3,i1 nB o2 t2 i1 nD o3 t2 i1 nD o4 t2 i0 || |s| nG o2 t2 i1 nB o2 t2 i1 nD o3 t2 i1 nD o4 t2 i0 || |s| nG o2 t2 i1 nB o2 t2 i1 nD o3 t2 i1 nD o4 t2 i0 || |s| nG o2 t2 i1 nB o2 t2 i1 nD o3 t2 i1 nD o4 t2 i0 || |e| |s|
4,nA o2 t2 i1 nC o3 t2 i1 nE o3 t2 i1 nC o5 t1 i0 || |s| nA o2 t2 i1 nC o3 t2 i1 nE o3 t2 i1 nC o5 t2 i0 || |s| nA o2 t2 i1 nC o3 t2 i1 nE o3 t2 i1 nA o3 t1 i0 || |s| nA o2 t2 i1 nC o3 t2 i1 nE o3 t2 i1 nA o3 t2 i0


In [9]:
ob = data.one_batch()

In [10]:
txt_out = data.vocab.textify(ob[0][0]).replace('xxbos ', ''); txt_out

'nF o3 t1 i0 nA o3 t1 i0 nC o4 t1 i0 || |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0 || |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0 || |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0 || |e| |s| nF o3 t2 i0 nA o3 t2 i0 nC o4 t2 i0'

In [11]:
from encode_data import *

In [19]:
a = data.train_ds[0][0]

In [30]:
atext = a.text

In [31]:
seq = str2seq(atext)

In [32]:
s = str2stream(atext)

In [33]:
s.show('midi')

In [36]:
s.show('text')

{0.0} <music21.stream.Part 0x7fb8186c9b70>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.key.KeySignature of no sharps or flats>
    {0.0} <music21.chord.Chord G3>
    {0.25} <music21.chord.Chord F#3>
    {0.5} <music21.chord.Chord A2>
    {0.75} <music21.chord.Chord F3>
    {1.0} <music21.chord.Chord E3>
    {1.25} <music21.chord.Chord A2>
    {1.5} <music21.chord.Chord F3>
    {1.75} <music21.chord.Chord E3>
    {2.0} <music21.chord.Chord A2>
    {2.75} <music21.chord.Chord G3>
    {3.0} <music21.chord.Chord F#3>
    {3.25} <music21.chord.Chord G3>
    {3.5} <music21.chord.Chord C#4>
{3.75} <music21.stream.Part 0x7fb818178b38>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.key.KeySignature of no sharps or flats>
    {0.0} <music21.chord.Chord E3 G3 B3>
    {4.0} <music21.chord.Chord E3 G3 A3 C4>
    {6.0} <music21.chord.Chord E3 G3 B3>
    {8.0} <music21.chord.Chord 

In [38]:
s.makeMeasures().show()

TypeError: append() argument must be xml.etree.ElementTree.Element, not Element

In [35]:
s.flat.show()

TypeError: append() argument must be xml.etree.ElementTree.Element, not Element